In [1]:
import findspark
findspark.init()
import pyspark

In [2]:
from pyspark.sql.functions import col, substring, round
from pyspark.sql.types import VarcharType, TimestampType, IntegerType, DoubleType

In [3]:
def calculate_distance(lat1, lon1, lat2, lon2):
    return ((lat2 - lat1)**2 + (lon2 - lon1)**2)**1/2

In [4]:
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder\
        .appName('fraud_detection')\
        .config("spark.jars", "E:\spark-3.5.0-bin-hadoop3\jars\mysql-connector-j-8.1.0.jar")\
        .getOrCreate()

In [5]:
trans = spark.read.format("csv").load("./train.csv", header='True', inferSchema='True')
cust = spark.read.format("csv").load("./customer.csv", header='True')

In [6]:
trans.printSchema()

root
 |-- cc_num: long (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- trans_date: timestamp (nullable = true)
 |-- trans_time: timestamp (nullable = true)
 |-- unix_time: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- merchant: string (nullable = true)
 |-- amt: integer (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)



In [7]:
cust.printSchema()

root
 |-- cc_num: string (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: string (nullable = true)



In [8]:
trans.show(5)

+----------------+-------+--------+--------------------+-------------------+-------------------+----------+--------------+--------------------+---+---------+----------+--------+
|          cc_num|  first|    last|           trans_num|         trans_date|         trans_time| unix_time|      category|            merchant|amt|merch_lat|merch_long|is_fraud|
+----------------+-------+--------+--------------------+-------------------+-------------------+----------+--------------+--------------------+---+---------+----------+--------+
|4641003399120410|Tiffany| Russell|a2de9ff6c581157f6...|2016-01-05 01:30:00|2024-01-23 20:54:01|1452027241|      misc_pos|            Wiza LLC|140|42.168126|-92.149976|       0|
|4738555317386146|   John|Peterson|6938e448d7a51744c...|2012-01-16 01:30:00|2024-01-23 14:46:57|1326725217|      misc_pos|         Block Group|230|34.899095|-87.758065|       0|
|    675985166411|Rebecca|   Brown|7aa4ecc3c69a223d4...|2016-07-31 01:30:00|2024-01-23 22:13:18|1470003198|hea

In [9]:
cust.show(5)

+---------------+----------+--------+------+--------------------+----------+-----+-----+-------+---------+--------------------+-------------------+
|         cc_num|     first|    last|gender|              street|      city|state|  zip|    lat|     long|                 job|                dob|
+---------------+----------+--------+------+--------------------+----------+-----+-----+-------+---------+--------------------+-------------------+
|101331974127885|Georgeanna| D'Aulby|     F|     6 Stuart Circle|Clearwater|   FL|34620|27.9139| -82.7157| Electrical Engineer|1946-04-03 00:00:00|
|103644429587006|     Davie|Chestnut|     M| 02 Crest Line Trail|   Visalia|   CA|93291|36.3551| -119.301| Marketing Assistant|1986-02-03 00:00:00|
|119517748522380|   Randall|  d' Eye|     M|    415 Towne Center|     Boise|   ID|83727|43.4599| -116.244|    Registered Nurse|1988-05-15 00:00:00|
|121937491260937|  Shepherd|  Dobble|     M|47668 Cottonwood ...| Las Vegas|   NV|89105| 36.086|-115.1471|Compen

In [10]:
joinedDF = trans.join(cust, on=['cc_num', 'first', 'last'], how='inner')

In [11]:
joinedDF.show(5)

+----------------+-------+--------+--------------------+-------------------+-------------------+----------+--------------+--------------------+---+---------+----------+--------+------+--------------------+--------+-----+-----+-------+--------+--------------------+-------------------+
|          cc_num|  first|    last|           trans_num|         trans_date|         trans_time| unix_time|      category|            merchant|amt|merch_lat|merch_long|is_fraud|gender|              street|    city|state|  zip|    lat|    long|                 job|                dob|
+----------------+-------+--------+--------------------+-------------------+-------------------+----------+--------------+--------------------+---+---------+----------+--------+------+--------------------+--------+-----+-----+-------+--------+--------------------+-------------------+
|4641003399120410|Tiffany| Russell|a2de9ff6c581157f6...|2016-01-05 01:30:00|2024-01-23 20:54:01|1452027241|      misc_pos|            Wiza LLC|14

In [12]:
joinedDF = joinedDF.withColumn("age", substring(col("trans_date"), 1, 4).cast("int") - substring(col("dob"), 1, 4).cast("int"))
joinedDF = joinedDF.withColumn("distance", round(calculate_distance(col("merch_lat").cast("double"), col("merch_long").cast("double"), col("lat").cast("double"), col("long").cast("double")), 5))


In [13]:
joinedDF.show(5)

+----------------+-------+--------+--------------------+-------------------+-------------------+----------+--------------+--------------------+---+---------+----------+--------+------+--------------------+--------+-----+-----+-------+--------+--------------------+-------------------+---+--------+
|          cc_num|  first|    last|           trans_num|         trans_date|         trans_time| unix_time|      category|            merchant|amt|merch_lat|merch_long|is_fraud|gender|              street|    city|state|  zip|    lat|    long|                 job|                dob|age|distance|
+----------------+-------+--------+--------------------+-------------------+-------------------+----------+--------------+--------------------+---+---------+----------+--------+------+--------------------+--------+-----+-----+-------+--------+--------------------+-------------------+---+--------+
|4641003399120410|Tiffany| Russell|a2de9ff6c581157f6...|2016-01-05 01:30:00|2024-01-23 20:54:01|1452027241

In [14]:
transDF = joinedDF.select("cc_num", "trans_num", "trans_date", "trans_time", "category", "amt", "merchant", "merch_lat", "merch_long", "age", "distance", "is_fraud")

In [15]:
# transDF = joinedDF.select(
#     (joinedDF.cc_num.cast(VarcharType(255))),
#     (joinedDF.trans_num.cast(VarcharType(255))),
#     (joinedDF.trans_date.cast(TimestampType())),
#     (joinedDF.trans_time.cast(TimestampType())),
#     (joinedDF.category.cast(VarcharType(255))),
#     (joinedDF.merchant.cast(VarcharType(255))),
#     (joinedDF.amt.cast(DoubleType())),
#     (joinedDF.merch_lat.cast(DoubleType())),
#     (joinedDF.merch_long.cast(DoubleType())),
#     (joinedDF.distance.cast(DoubleType())),
#     (joinedDF.age.cast(IntegerType())),
#     (joinedDF.is_fraud.cast(IntegerType()))
# )

In [16]:
transDF.show(5)

+----------------+--------------------+-------------------+-------------------+--------------+---+--------------------+---------+----------+---+--------+--------+
|          cc_num|           trans_num|         trans_date|         trans_time|      category|amt|            merchant|merch_lat|merch_long|age|distance|is_fraud|
+----------------+--------------------+-------------------+-------------------+--------------+---+--------------------+---------+----------+---+--------+--------+
|4641003399120410|a2de9ff6c581157f6...|2016-01-05 01:30:00|2024-01-23 20:54:01|      misc_pos|140|            Wiza LLC|42.168126|-92.149976| 39|  1.5E-4|       0|
|4738555317386146|6938e448d7a51744c...|2012-01-16 01:30:00|2024-01-23 14:46:57|      misc_pos|230|         Block Group|34.899095|-87.758065| 30|  3.0E-4|       0|
|    675985166411|7aa4ecc3c69a223d4...|2016-07-31 01:30:00|2024-01-23 22:13:18|health_fitness| 58|          Heller PLC|44.771892|-75.124947| 41|  1.4E-4|       0|
|5264302655249852|738f

In [17]:
transDF.printSchema()

root
 |-- cc_num: long (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- trans_date: timestamp (nullable = true)
 |-- trans_time: timestamp (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: integer (nullable = true)
 |-- merchant: string (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- distance: double (nullable = true)
 |-- is_fraud: integer (nullable = true)



In [18]:
# Write to MySQL Table
transDF.write \
    .format("jdbc") \
    .option("driver","com.mysql.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard") \
    .option("dbtable", "transaction") \
    .option("user", "root") \
    .option("password", "khang") \
    .mode("overwrite") \
    .save()

In [ ]:
# Write to MySQL Table
cust.write \
    .format("jdbc") \
    .option("driver","com.mysql.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard") \
    .option("dbtable", "customer") \
    .option("user", "root") \
    .option("password", "khang") \
    .mode("overwrite") \
    .save()